In [1]:
# Step 1: Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Step 2: Generate Synthetic Dataset (Circuit Parameters)
# 6 input features: voltage, current, resistance, temp, freq, phase_shift
X, y = make_classification(
    n_samples=2000,
    n_features=6,
    n_informative=5,
    n_redundant=0,
    n_clusters_per_class=1,
    weights=[0.7, 0.3],   # 70% normal, 30% faulty
    class_sep=1.5,
    random_state=42
)

columns = ['voltage', 'current', 'resistance', 'temperature', 'frequency', 'phase_shift']
df = pd.DataFrame(X, columns=columns)
df['fault_status'] = y

print("Dataset Sample:")
print(df.head())


Dataset Sample:
    voltage   current  resistance  temperature  frequency  phase_shift  \
0  1.924638 -1.477349   -1.767829     1.022021   1.300660     0.447971   
1  2.065081  0.044026   -2.329929     1.202002   0.695157     1.439622   
2  1.670069 -0.032629    1.856094    -2.171573   2.547395     0.463890   
3  1.492829  1.715700    1.439215    -1.295311   4.472366     4.625293   
4  0.807903  2.204851    1.257488    -0.341218   1.133169     1.300668   

   fault_status  
0             0  
1             0  
2             1  
3             1  
4             1  


In [3]:
# Step 3: Data Preprocessing
X = df.drop('fault_status', axis=1)
y = df['fault_status']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTraining set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (1600, 6)
Testing set size: (400, 6)


In [4]:
# Step 4: Build ANN Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Step 5: Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [6]:
# Step 6: Train Model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7094 - loss: 0.6321 - val_accuracy: 0.9844 - val_loss: 0.3385
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9891 - loss: 0.2497 - val_accuracy: 1.0000 - val_loss: 0.0562
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9915 - loss: 0.0692 - val_accuracy: 1.0000 - val_loss: 0.0225
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9922 - loss: 0.0594 - val_accuracy: 1.0000 - val_loss: 0.0140
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9898 - loss: 0.0595 - val_accuracy: 0.9969 - val_loss: 0.0117
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9935 - loss: 0.0438 - val_accuracy: 1.0000 - val_loss: 0.0086
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9918 - loss: 0.0540 - val_accuracy: 0.9969 - val_loss: 0.0112
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9923 - loss: 0.0531 - val_accuracy: 0.9969 - val_loss:

In [7]:
# Step 7: Evaluate Model on Test Data
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [8]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       279
           1       1.00      0.98      0.99       121

    accuracy                           0.99       400
   macro avg       1.00      0.99      0.99       400
weighted avg       1.00      0.99      0.99       400

